In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from tqdm.notebook import tqdm


import json
import os
import time
import csv

from objects import song_artist
from parallelization import parallel
from methods import table_noGen
from methods import table_artGen

import multiprocessing
from joblib import Parallel, delayed

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import matplotlib.pyplot as plt
plt.rc('font', size=14)
import seaborn as sns
sns.set(style='whitegrid', color_codes=True, rc={'figure.figsize':(11,8)}, font_scale=2)

In [2]:
#os.chdir('/c/Users/matth/Documents/Coding/spotify/')

In [3]:
#!pwd

## Data Generation Process for Spotify Genres Project
### Initial Data Importation and Prep

In [4]:
# read streaming data and concat rows
strm_base = pd.DataFrame()
for file in os.listdir("./data"):
    if file.startswith("Streaming"):
        file_path = "./data/" + file
        temp = pd.read_json(file_path)
        strm_base = pd.concat([strm_base,temp])
        

strm_base.index = range(0,strm_base.shape[0])
strm_base = strm_base.astype({'endTime': 'datetime64','artistName':'string','trackName':'string'})

In [5]:
# adding song_artist object in preparation for parallel feature addition
strm_base['song_artist'] = strm_base.apply(lambda x: song_artist(x['trackName'], x['artistName']), axis=1)
data = strm_base['song_artist'].tolist()

In [6]:
cols = ['track_id', 'danceability','energy', 'key', 'loudness', 'mode',
        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
        'valence', 'tempo', 'duration_ms', 'time_signature']

In [7]:
file = open('./modified/feats_table.csv', 'w')
writer = csv.writer(file)
writer.writerow(cols)
file.close()

### Parallelized Audio Features

In [ ]:
start_time = time.time()

parallel(data,table_noGen)

print("Table creation took", time.time()-start_time, "to run") # 

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'The Restructuring of the LCS Players Association feat. Jacob Wolf FTW with Imad Khan: An Esports and Competitive Gaming Podcast', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'The Significance of the Sinatraa Suspension feat. Hunter Cooke FTW with Imad Khan: An Esports and Competitive Gaming Podcast', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'The Restructuring of the LCS Players Association feat. Jacob Wolf FTW with Imad Khan: An Esports and Competitive Gaming Podcast', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Crazy Noisy Bizarre Town (From "Jojo\'s

In [ ]:
feats_table = pd.read_csv('./modified/feats_table.csv')
print(feats_table.shape[0] == strm_base.shape[0])
print(feats_table.index == strm_base.index)

In [ ]:
feats_table.info()

In [ ]:
feats_table.index = range(0,feats_table.shape[0])
temp = pd.concat([strm_base,feats_table], axis = 0)

### Imputing Artist Genres

In [ ]:
art_names = temp['artistName'].unique().tolist()
temp['genre'] = temp['artistName']

In [ ]:
start_time = time.time()

artGen_dict = {artist: table_artGen(artist) for artist in art_names}

print("My program took", time.time() - start_time, "to run") # 

In [ ]:
temp['genre'].replace(artGen_dict, inplace=True)

In [ ]:
temp.info()

### Exporting Final Table

In [ ]:
final = temp.dropna()

In [ ]:
final.to_csv(path_or_buf = './final/final_table.csv', index = False)